# Regressão Logística I
## Tarefa II

Vamos trabalhar com a mesma base do exercício anterior, mas vamos aprofundar um pouco mais a nossa regressão.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.formula.api as smf

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

df = pd.read_csv(url, 
                 names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                        'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'])
df['flag_doente'] = (df['num']!=0).astype('int64')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0,0


A descrição das variáveis está recortada abaixo:
- age: idade do paciente em anos
- sex: sexo (1 = male; 0 = female)  
- cp: tipo de dor no peito
  - 1: angina típica
  - 2: angina atípica
  - 3: dor não-angina
  - 4: assintomático
- trestbps: pressão sanguínea em repouso (em mm Hg na admissão ao hospital
- chol: colesterol sérico em mg/dl
- fbs: (açúcar no sangue em jejum > 120 mg/dl) (1 = True; 0 = False)
- restecg: resultados eletrocardiográficos em repouso
  - 0: normal
  - 1: tendo anormalidade da onda ST-T (Inversões de onda T e / ou ST com elevação ou depressão de > 0.05 mV)
  - 2: mostrando hipertrofia ventricular esquerda provável ou definitiva pelos critérios de Estes
- thalach: frequência cardíaca máxima alcançada
- exang: angina induzida por exercício(1 = sim; 0 = não)
- oldpeak = Depressão de ST induzida por exercício em relação ao repouso
- slope: Depressão de ST induzida por exercício em relação ao repouso
  - 1: inclinação ascendente
  - 2: estável
  - 3: inclinação descendente
- ca: número de vasos principais (0-3) coloridos por fluorosopia
- thal: 3 = normal; 6 = defeito corrigido; 7 = defeito reversível
- num: diagnóstico de doença cardíaga (status de doença angiográfica)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          303 non-null    float64
 1   sex          303 non-null    float64
 2   cp           303 non-null    float64
 3   trestbps     303 non-null    float64
 4   chol         303 non-null    float64
 5   fbs          303 non-null    float64
 6   restecg      303 non-null    float64
 7   thalach      303 non-null    float64
 8   exang        303 non-null    float64
 9   oldpeak      303 non-null    float64
 10  slope        303 non-null    float64
 11  ca           303 non-null    object 
 12  thal         303 non-null    object 
 13  num          303 non-null    int64  
 14  flag_doente  303 non-null    int64  
dtypes: float64(11), int64(2), object(2)
memory usage: 35.6+ KB


1. Considere o script que monta a análise bivariada que você fez na tarefa anterior. Transforme esse script em uma função, que deve:
- Ter como parâmetros de entrada:
    - Um *dataframe* contendo os dados a serem avaliados
    - Um *string* contendo o nome da variável resposta
    - Um *string* contendo o nome da variável explicativa
- E deve retornar um *dataframe* com os dados da bivariada. 
**Monte** a mesma bivariada pelo menos três variáveis qualitativas do *data-frame*. Qual delas parece discriminar mais o risco?

In [30]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0,0


In [36]:
def script_analise_biv(nome_data, resposta, explicativa):
    nome_data = pd.crosstab(df[resposta], df[explicativa], margins=True)
    nome_data["Media_de_doentes"] = nome_data[1]/nome_data["All"]
    nome_data["chance"] = nome_data[1]/nome_data[0]
    nome_data.loc["All", "chance"]
    nome_data["RC_vs_All"] = nome_data["chance"]/nome_data.loc["All", "chance"]
    return nome_data

In [37]:
novo_data = script_analise_biv("data_frame_ex", "sex", "flag_doente")
novo_data

flag_doente,0,1,All,Media_de_doentes,chance,RC_vs_All
sex,,,,,,
0.0,72,25,97,0.257732,0.347222,0.409672
1.0,92,114,206,0.553398,1.239130,1.461996
All,164,139,303,0.458746,0.847561,1.000000


In [44]:
reglog = smf.logit("1 ~ Media_de_doentes + chance + RC_vs_All", data=novo_data).fit()

reglog.summary()

         Current function value: 0.000000
         Iterations: 35


C:\Users\victo\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\victo\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\victo\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\victo\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
C:\Users\victo\anaconda3

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Intercept   No. Observations:                    3
Model:                          Logit   Df Residuals:                        0
Method:                           MLE   Df Model:                            2
Date:                Tue, 05 Mar 2024   Pseudo R-squ.:                     inf
Time:                        03:30:37   Log-Likelihood:            -2.2204e-15
converged:                      False   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           28.6294        nan        nan        nan         nan         nan
Media_de_doentes     3.7554        nan        nan        nan         nan         nan
chance               4.9388        nan        nan        nan         nan         nan
RC_vs_All            5.8271   3.68e+15   1.58e-15      1.000   -7.21e+15    7.21e+15
====================================================================================

Complete Separation: The results show that there iscomplete separation or perfect prediction.
In this case the Maximum Likelihood Estimator does not exist and the parameters
are not identified.
"""

In [45]:
print("A que parece discriminar mais o risco é a variável RC_vs_All porque é a única que apareceram os dados.")

A que parece discriminar mais o risco é a variável RC_vs_All porque é a única que apareceram os dados.


2. Monte uma função semelhante para categorizar variáveis quantitativas contínuas (com muitas categorias) como ```age```.  
    Além dos mesmos parâmetros da função anterior, defina mais um parâmetro como número de categorias que você deseja quebrar. Defina um valor '*default*' de 5 grupos para este parâmetro.  

In [56]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,flag_doente
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0,0


In [67]:
def exercicio_num_2(nome_data, resposta, explicativa, num_grupos=5):
    if num_grupos == 5:
        df.loc[(df["resposta"] >= 0) & (df["resposta"] <= 12), "nivel_idade"] = "Criança"
        df.loc[(df["resposta"] >= 13) & (df["resposta"] <= 17), "nivel_idade"] = "Adolescente"
        df.loc[(df["resposta"] >= 18) & (df["resposta"] <= 40), "nivel_idade"] = "Adulto"
        df.loc[(df["resposta"] >= 41) & (df["resposta"] <= 55), "nivel_idade"] = "Melhor idade"
        df.loc[(df["resposta"] >= 56) & (df["resposta"] <= 100), "nivel_idade"] = "Idoso"
        nome_data = pd.crosstab(df["age"], df["nivel_idade"], margins=True)
        nome_data["Quantidade_media_adulto"] = nome_data["Adulto"]/nome_data["All"]
        nome_data["Quantidade_media_idoso"] = nome_data["Idoso"]/nome_data["All"]
        nome_data["Quantidade_media_melhor_idade"] = nome_data["Melhor idade"]/nome_data["All"]
    elif num_grupos == 4:
        df.loc[(df["resposta"] >= 13) & (df["resposta"] <= 17), "nivel_idade"] = "Adolescente"
        df.loc[(df["resposta"] >= 18) & (df["resposta"] <= 40), "nivel_idade"] = "Adulto"
        df.loc[(df["resposta"] >= 41) & (df["resposta"] <= 55), "nivel_idade"] = "Melhor idade"
        df.loc[(df["resposta"] >= 56) & (df["resposta"] <= 100), "nivel_idade"] = "Idoso"
        nome_data = pd.crosstab(df["resposta"], df["nivel_idade"], margins=True)
        nome_data["Quantidade_media_adulto"] = nome_data["Adulto"]/nome_data["All"]
        nome_data["Quantidade_media_idoso"] = nome_data["Idoso"]/nome_data["All"]
        nome_data["Quantidade_media_melhor_idade"] = nome_data["Melhor idade"]/nome_data["All"]
        
        return nome_data

In [69]:
novo = exercicio_num_2("data_frame_ex", "age", "flag_doente", 4)
novo

KeyError: 'resposta'

3. Construa um modelo de regressão logística com as variáveis qualitativas: ```sex + cp +  trestbps``` e com a variável quantitativa ```age```.

**Interprete os parâmetros.**

4. Avalie o seu modelo quanto a **calibragem**:
- Calcule a probabilidade de evento predita segundo o seu modelo
- Categorize essa probabilidade em G=5 grupos
- Calcule a probabilidade de evento predita média por grupo
- Calcule a taxa de eventos (média da variável indicadora de eventos) por grupo
- Compare graficamente o valor eperado versus observado para a taxa de maus por grupo

5. Avalie o seu modelo quanto a discriminação calculando acurácia, GINI e KS.

6. tente melhorar o modelo obtido, por exemplo inserindo ou removendo variáveis.  
    Avalie as características do seu modelo (calibragem e acurácia).